In [16]:
import os
import re
import shutil
import argparse
import numpy as np
import pandas as pd
import nltk
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [2]:
def read_data(dir_path):
    if dir_path is None:
        return None
    files = os.listdir(dir_path)
    records =[]
    rurls = []
    labels = []
    for file in files:
        with open(os.path.join(dir_path, file), 'r') as f:
            for line in f:
                splits = line.strip().split('\t')
                record = re.sub('[^A-Za-z_]', '', splits[1])
                record = re.sub(' +', ' ', record.replace('_', ' ').strip())
                if len(record) > 1:
                    records.append(record)
                    rurls.append(splits[0])
                    labels.append(splits[2])
    return rurls, records, labels

In [3]:
def convert_list_of_tuple_to_string(lst=None):
    if lst is None:
        return None
    lst = ['_'.join(item) for item in lst]
    return ' '.join(lst)

In [4]:
def extract_bigram_dataframe(records=None):
    if records is None:
        return None
    bigrams_lists = [list(nltk.bigrams(record.split())) for record in records]
    texts = [convert_list_of_tuple_to_string(lst) for lst in bigrams_lists]
    bigrams = list(itertools.chain.from_iterable(bigrams_lists))
    vocab = list(set(['_'.join(bigram) for bigram in bigrams]))
    print("The feature size is:")
    print(len(vocab))
    cv = CountVectorizer(vocabulary=vocab)
    df = pd.DataFrame({'record':records, 'text':texts})
    df = pd.concat([df, pd.SparseDataFrame(cv.fit_transform(df['text']), df.index, cv.get_feature_names(), default_fill_value=0)], axis=1) 
    return df;
    
    

In [5]:
def split_train_test_validation_data(data=None, test_size=None, validation_size=None, random_state=None):
    data = data.copy()
    y = data['label']
    X = data.drop(['rurl', 'record', 'text', 'label'], axis=1)
    X, X_test, y, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size, random_state=random_state)
    return X_train, y_train, X_validation, y_validation, X_test, y_test

In [6]:
def preprocessor(data_path=None):
    if data_path is None:
        return None
    rurls, records, labels = read_data(data_path)
    data = extract_bigram_dataframe(records)
    data.insert(0, 'rurl', rurls)
    data['label'] = labels
    return split_train_test_validation_data(data, 0.2, 0.25)

In [13]:
X_train, y_train, X_validation, y_validation, X_test, y_test = preprocessor('test')

The feature size is:
26


NameError: name 'train_test_split' is not defined

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
rurls, records, labels = read_data('test')


bigrams_lists = [list(nltk.bigrams(record.split())) for record in records]
texts = [convert_list_of_tuple_to_string(lst) for lst in bigrams_lists]
bigrams = list(itertools.chain.from_iterable(bigrams_lists))
vocab = list(set(['_'.join(bigram) for bigram in bigrams]))

cv = CountVectorizer(vocabulary=vocab)

#tokens = ['_'.join(item) for item in bigram_list for bigram_list in bigrams_lists]
print(records)
print(labels)
print(bigrams_lists)
print(len(records))
print(list(texts))
print(list(tokens))
print(vocab)

In [ ]:
import nltk
X = 'clothing true apparels crew general clothes paid search brand'
Y = list(nltk.bigrams(X.split()))
print(Y)
string_list = ['_'.join(item) for item in Y]
print(string_list)

In [ ]:
import nltk
X = 'clothing true apparels crew general clothes paid search brand'
Y = list(nltk.bigrams(X.split()))
print(Y)
string_list = convert_list_of_tuple_to_string(Y)
print(string_list)